# Detecting BDM In Superbowl Commercials

In [1]:

%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os

In [3]:
INDUSTRY_SPECIFIC_AWARENESS = True
BRAND_SPECIFIC_AWARENESS = True

# Donwload Ads


In [4]:
def download_videos():
# Define the URL and the target directory
  url = 'https://box.fu-berlin.de/s/zwxKp8PXkCwAwGe/download'
  download_filename = 'downloaded_archive.zip'
  target_directory = 'ADs'

  # Step 1: Download the file
  os.system(f'wget -O {download_filename} {url}')

  # Step 2: Create the target directory if it doesn't exist
  os.makedirs(target_directory, exist_ok=True)

  # Step 3: Extract the downloaded archive to the target directory
  os.system(f'unzip -o {download_filename} -d {target_directory}')

  # Step 4: Remove the downloaded file
  os.remove(download_filename)

  print(f"Archive extracted to {target_directory} and {download_filename} removed.")

# Comment in to download the videos
# download_videos()

# Merge Excel Files

In [5]:
BDM_excel = pd.read_excel('BDM.xlsx')
final_excel = pd.read_excel('previous_project_results.xlsx')

In [6]:
final_excel = final_excel.merge(
    BDM_excel[['AdNumber', 'BDM']], 
    on='AdNumber', 
    how='left',
    suffixes=('_old', '')
).drop('BDM_old', axis=1, errors='ignore')


# print number of rows where BDM is NaN, 0 and 1
print(f"Number of rows where BDM is NaN: {final_excel[final_excel['BDM'].isna()].shape[0]}")
print(f"Number of rows where BDM is 0: {final_excel[final_excel['BDM'] == 0].shape[0]}")
print(f"Number of rows where BDM is 1: {final_excel[final_excel['BDM'] == 1].shape[0]}")


Number of rows where BDM is NaN: 22
Number of rows where BDM is 0: 411
Number of rows where BDM is 1: 139


In [7]:
ad_df = final_excel.groupby(['cont_primary_product_type', 'BRAND', 'AdNumber', "BDM"]).size().reset_index(name='count')
ad_df.rename(columns={'cont_primary_product_type': 'product_category', 'BRAND': 'brand', 'AdNumber': 'commercial_number'}, inplace=True)
ad_df.drop(columns=['count'], inplace=True)
ad_df.head(10)

,product_category,brand,commercial_number,BDM
0,1.0,AvocadosfromMexico,AD0357,0.0
1,1.0,AvocadosfromMexico,AD0414,1.0
2,1.0,AvocadosfromMexico,AD0474,0.0
3,1.0,AvocadosfromMexico,AD0525,1.0
4,1.0,AvocadosfromMexico,AD0584,1.0
5,1.0,AvocadosfromMexico,AD0635,1.0
6,1.0,AvocadosfromMexico,AD0745,0.0
7,1.0,BUBLY,AD0586,1.0
8,1.0,Bai,AD0475,0.0
9,1.0,Butterfinger,AD0420,1.0


### Brand Keywords

In [8]:
if BRAND_SPECIFIC_AWARENESS:
    product_brand_df = pd.read_csv("product_brands.csv")

    product_brand_df['brand'] = product_brand_df['brand'].str.replace(' ', '').str.lower()
    ad_df['brand_clean'] = ad_df['brand'].str.replace(' ', '').str.lower()

    # Merge the dataframes
    ad_df = ad_df.merge(
        product_brand_df[['brand', 'product_brand_keywords']], 
        left_on='brand_clean',
        right_on='brand',
        how='left',
        suffixes=('', '_brand')
    )

    # Clean up columns
    ad_df.drop(['brand_clean', 'brand_brand'], axis=1, inplace=True)


    ad_df.head(10)

### Product Category Keywords

In [9]:
if INDUSTRY_SPECIFIC_AWARENESS:
    product_brands_df = pd.read_csv("product_categories.csv")
    product_brands_df.head(40)
    product_brands_df = product_brands_df.drop('product_cat_id', axis=1)
    ad_df = ad_df.drop('product_category', axis=1)
    display(product_brands_df)
    display(ad_df)

    # Create a dictionary to map brands to their product categories and other attributes
    brand_to_info = {}
    for _, row in product_brands_df.iterrows():
        # Convert string representation of list to actual list
        brands = eval(row['product_cat_brands'])
        for brand in brands:
            # Remove spaces and convert to lowercase for more robust matching
            brand = brand.replace(' ', '').lower()
            # Store all columns for this brand
            brand_to_info[brand] = {col: row[col] for col in product_brands_df.columns}

    # Function to find category info for a brand
    def find_brand_info(brand):
        # Clean brand name for matching
        clean_brand = brand.replace(' ', '').lower()
        return brand_to_info.get(clean_brand)

    # Add all product category columns to ad_df
    for col in product_brands_df.columns:
        ad_df[col] = ad_df['brand'].apply(lambda x: find_brand_info(x)[col] if find_brand_info(x) else None)

    # Print brands that couldn't be mapped
    unmapped_brands = ad_df[ad_df['product_cat_name'].isna()]['brand'].unique()
    if len(unmapped_brands) > 0:
        print("Brands without category mapping:")
        for brand in unmapped_brands:
            print(f"- {brand}")

    # Print number of rows with missing category
    print(f"Number of rows where product category is NaN: {ad_df[ad_df['product_cat_name'].isna()].shape[0]}")


    print(f"Final number of rows with missing categories: {ad_df[ad_df['product_cat_name'].isna()].shape[0]}")

    ad_df.head(10)

,product_cat_name,product_cat_keywords,product_cat_brands
0,Alcoholic beverages (Beer Hard Seltzer),"['smooth', 'rich', 'refreshing', 'aromatic', '...","['Anheuser Busch InBev', 'Becks Beer', 'Bud Li..."
1,Banking & Investments,"['secure', 'reliable', 'customized', 'personal...","['BankofAmerica', 'Coinbase', 'Cryptocom', 'Di..."
2,Car Accessories & Related Services,"['safe', 'protect', 'protected', 'protection',...","['Michelin', 'Wallbox', 'Weathertech']"
3,Car Manufacturer,"['Luxurious', 'Efficient', 'Powerful', 'Innova...","['Acura', 'AlfaRomeo', 'Audi', 'BMW', 'Buick',..."
4,Car Sales & Services Platform,"['easy', 'perfect', 'love', 'expert', 'really'...","['Carvana', 'Vroom', 'Carscom', 'Carmax']"
5,"Clothing, shoes and apparel","['favorite', 'designed', 'comfortable', 'uncom...","['CalvinKlein', 'Gildan', 'HANDM', 'Marmot', '..."
6,Consumer Electronics and Appliances,"['Smart', 'Display', 'Control', 'Touchscreen',...","['BlackBerry', 'Google', 'Intel', 'LGEEAudioVi..."
7,Cosmetics and personal care products,"['luxurious', 'nourishing', 'radiant', 'smooth...","['Fitbit', 'PlanetFitness']"
8,Diet and exercise products,"['healthy', 'energizing', 'effective', 'smart'...","['CalvinKlein', 'Gildan', 'Marmot', 'Skechers'..."
9,Food Delivery Services,"['easy', 'delivered', 'local', 'convenience', ...","['DoorDash', 'UberEats']"


,brand,commercial_number,BDM,product_brand_keywords
0,AvocadosfromMexico,AD0357,0.0,"always in season', 'taste', 'delicious', 'good..."
1,AvocadosfromMexico,AD0414,1.0,"always in season', 'taste', 'delicious', 'good..."
2,AvocadosfromMexico,AD0474,0.0,"always in season', 'taste', 'delicious', 'good..."
3,AvocadosfromMexico,AD0525,1.0,"always in season', 'taste', 'delicious', 'good..."
4,AvocadosfromMexico,AD0584,1.0,"always in season', 'taste', 'delicious', 'good..."
...,...,...,...,...
545,Wixcom,AD0408,0.0,NaN
546,Wixcom,AD0465,0.0,NaN
547,Wixcom,AD0522,0.0,NaN
548,Wixcom,AD0579,0.0,NaN


Brands without category mapping:
- AvocadosfromMexico
- Beck's Beer
- Hyndai
- Volkswagen
- LGCEAudioVisual
- Always
- Axe
- Colgate
- DollarShaveClub
- DoveMenCare
- DrSquatch
- Gillette
- Huggies
- IrishSpring
- Itsa10Haircare
- Olay
- Schick
- Weightwatchers
- UNIVERSALPARKSANDRESORTS
- BassProShops
- THEWASHINGTONPOST
- Mercari
- CaesarsSportsbook
- Century21
- Fiverr
- Groupon
- Indeed
Number of rows where product category is NaN: 38
Final number of rows with missing categories: 38


In [10]:
ad_df = ad_df.dropna()
# TODO: Implement proper handling of missing values

# INFO: COMMENT IN THIS SECTION FOR DEBUGGING
if BRAND_SPECIFIC_AWARENESS or INDUSTRY_SPECIFIC_AWARENESS:
  ad_df = ad_df.head(20)
else:
  ad_df = ad_df.head(20)
display(ad_df)

# save progress so far to csv
# make csv dir if it doesn't exist
os.makedirs('csvs', exist_ok=True)
ad_df.to_csv('csvs/1_ad_df_baseline.csv', index=False)

,brand,commercial_number,BDM,product_brand_keywords,product_cat_name,product_cat_keywords,product_cat_brands
59,MountainDew,AD0442,1.0,"juice', 'caffeine', 'zero', 'sugar', 'refreshi...",Softdrinks,"['refreshing', 'organic', 'fizzy', 'sparkling'...","['Bai', 'BUBLY', 'CocaCola', 'DietCoke', 'MiO'..."
60,MountainDew,AD0533,0.0,"juice', 'caffeine', 'zero', 'sugar', 'refreshi...",Softdrinks,"['refreshing', 'organic', 'fizzy', 'sparkling'...","['Bai', 'BUBLY', 'CocaCola', 'DietCoke', 'MiO'..."
61,MountainDew,AD0659,1.0,"juice', 'caffeine', 'zero', 'sugar', 'refreshi...",Softdrinks,"['refreshing', 'organic', 'fizzy', 'sparkling'...","['Bai', 'BUBLY', 'CocaCola', 'DietCoke', 'MiO'..."
62,MountainDew,AD0717,0.0,"juice', 'caffeine', 'zero', 'sugar', 'refreshi...",Softdrinks,"['refreshing', 'organic', 'fizzy', 'sparkling'...","['Bai', 'BUBLY', 'CocaCola', 'DietCoke', 'MiO'..."
87,Snickers,AD0382,1.0,"hungry', 'satisfies', 'eat', 'hostile', 'crank...",Snacks,"['artificial', 'zero', 'refreshing', 'organic'...","['Butterfinger', 'Cheerios', 'Cheetos', 'Choba..."
88,Snickers,AD0453,1.0,"hungry', 'satisfies', 'eat', 'hostile', 'crank...",Snacks,"['artificial', 'zero', 'refreshing', 'organic'...","['Butterfinger', 'Cheerios', 'Cheetos', 'Choba..."
89,Snickers,AD0505,0.0,"hungry', 'satisfies', 'eat', 'hostile', 'crank...",Snacks,"['artificial', 'zero', 'refreshing', 'organic'...","['Butterfinger', 'Cheerios', 'Cheetos', 'Choba..."
90,Snickers,AD0673,0.0,"hungry', 'satisfies', 'eat', 'hostile', 'crank...",Snacks,"['artificial', 'zero', 'refreshing', 'organic'...","['Butterfinger', 'Cheerios', 'Cheetos', 'Choba..."
100,Bud Light,AD0259,0.0,"rice', 'hops', 'corn syrup', 'America', 'taste...",Alcoholic beverages (Beer Hard Seltzer),"['smooth', 'rich', 'refreshing', 'aromatic', '...","['Anheuser Busch InBev', 'Becks Beer', 'Bud Li..."
101,BudLight,AD0306,0.0,"rice', 'hops', 'corn syrup', 'America', 'taste...",Alcoholic beverages (Beer Hard Seltzer),"['smooth', 'rich', 'refreshing', 'aromatic', '...","['Anheuser Busch InBev', 'Becks Beer', 'Bud Li..."


## Retrieving Transcript

In [11]:
ad_df = pd.read_csv('csvs/1_ad_df_baseline.csv')

In [ ]:
import glob
from pathlib import Path
from transcript import transcribe_video
from ocr import ocr
ads_dir = "ADs"

def find_video_file(commercial_number, ads_dir):
    """Find the video file path for a given commercial number."""
    # Search recursively for MP4 files
    pattern = f"{ads_dir}/**/{commercial_number}.mp4"
    matches = glob.glob(pattern, recursive=True)
    return matches[0] if matches else None

# Create transcript column
ad_df['transcript'] = ''

# Process each commercial
for idx, row in ad_df.iterrows():
    commercial_number = row['commercial_number']
    video_path = find_video_file(commercial_number, ads_dir)
    
    if video_path:
        transcript = transcribe_video(video_path)
        # INFO: Uncomment the line below for debugging
        # ocr = 'Not Yet Implemented yet'
        ocr_text = ocr(video_path)
        ad_df.at[idx, 'transcript'] = transcript
        ad_df.at[idx, 'audio_only_transcript'] = transcript
        ad_df.at[idx, 'ocr_text'] = ocr
    else:
        print(f"Video not found for commercial {commercial_number}")

ad_df[ad_df['transcript'].notna()]
ad_df["audio_only_transcript"] = ad_df["transcript"]
ad_df.head(10)

/usr/local/lib/python3.9/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/lib

cleaned_text: 9b START Sat THREE AWESOME THINGS COMBINED DEW Kick
Video not found for commercial AD0533


/usr/local/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a G

cleaned_text: 7459 T72 1EZ 11Z22 en MEwI AS GOOD AS THE ORIGINAL DEW DEW and the DEW Logo are registered trademarks of Inc DZ erg


/usr/local/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a G

...

/usr/local/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a G

cleaned_text: 8243 8248 238 Bar ED 88 LISNT Gd 1 ta ITS ONLY WEIRD IF IT DOESN'T WORK BUD ENJOY RESPONSIBLY 2013 BUD LIGHT ST LOUIS MO


/usr/local/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a G

cleaned_text: BUD Bu Gi BL COLD COLD Bub Bub La 45 now for Afrojack 0 world Track tor Alrojack's re SHAZAM Afrojack rs 34 DISTINCTIVE DESIGN COLDER THAN COLD Tad ALUMINUM Ae ALUMINUM Bub DG 93 BU BUD OPEN UP SOMETHING NEW Sid THE NEW COOL Twist 1602 BOTTLE Bud ENJOY
Video not found for commercial AD0307


/usr/local/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a G

In [ ]:

ad_df.to_csv('csvs/2_ad_df_transcript.csv', index=False)

## Adding OCR Text

In [ ]:
ad_df = pd.read_csv('csvs/2_ad_df_transcript.csv')

In [ ]:

from ocr import ocr

In [ ]:
!python -m spacy download en_core_web_sm


# Determining Frequency of Superlatives and Comparative Adjectives

In [ ]:

ad_df = pd.read_csv('csvs/3_ad_df_ocr.csv')

In [ ]:
import spacy
import pandas as pd
from collections import Counter
import text_analysis as ta

# Create new columns to store the counts and percentages
ad_df['word_count'] = 0
ad_df['superlative_count'] = 0
ad_df['comparative_count'] = 0
ad_df['uniqueness_count'] = 0
ad_df['superlative_pct'] = 0.0
ad_df['comparative_pct'] = 0.0
ad_df['uniqueness_pct'] = 0.0
ad_df['total_bdm_terms_count'] = 0
ad_df['total_bdm_terms_pct'] = 0.0

# Process each row
for idx, row in ad_df.iterrows():
    word_count = len(ta.get_tokens(row['transcript']))
    ad_df.at[idx, 'word_count'] = word_count

    superlatives = ta.get_superlatives(row['transcript'])
    ad_df.at[idx, 'superlatives'] = ', '.join(superlatives) if superlatives else ''
    superlative_count = len(superlatives) if superlatives else 0
    ad_df.at[idx, 'superlative_count'] = superlative_count

    comparatives = ta.get_comparatives(row['transcript'])
    ad_df.at[idx, 'comparatives'] = ', '.join(comparatives) if comparatives else ''
    comparative_count = len(comparatives) if comparatives else 0
    ad_df.at[idx, 'comparative_count'] = comparative_count
    
    unique_words = ta.get_unique_words(row['transcript'])
    ad_df.at[idx, 'unique_words'] = ', '.join(unique_words) if unique_words else ''
    uniqueness_count = len(unique_words) if unique_words else 0
    ad_df.at[idx, 'uniqueness_count'] = uniqueness_count

    if word_count > 0:
        ad_df.at[idx, 'superlative_pct'] = superlative_count / word_count * 100
        ad_df.at[idx, 'comparative_pct'] = comparative_count / word_count * 100
        ad_df.at[idx, 'uniqueness_pct'] = uniqueness_count / word_count * 100
        
        total_bdm_terms = superlative_count + comparative_count + uniqueness_count
        ad_df.at[idx, 'total_bdm_terms_count'] = total_bdm_terms
        ad_df.at[idx, 'total_bdm_terms_pct'] = total_bdm_terms / word_count * 100

# Sort the DataFrame
ad_df = ad_df.sort_values(
    by=['superlative_count', 'comparative_count', 'superlative_pct', 'comparative_pct', 'uniqueness_pct'],
    ascending=[False, False, False, False, False]
)

# Display top 10 results
ad_df.head(10)

In [ ]:
# remove superlatives, comparatives and unique_words from ad_df
ad_df.drop(columns=['comparative_pct', 'superlative_pct', 'uniqueness_pct'], inplace=True)
ad_df.head(10)


## Nomen + Adjektive

In [ ]:
# Apply the function to each transcript individually using pandas apply
ad_df["adj_noun_pairs"] = ad_df["transcript"].apply(ta.extract_adj_noun_pairs)

# Add a column for the number of adjective-noun pairs
ad_df["num_adj_noun_pairs"] = ad_df["adj_noun_pairs"].apply(len)

# Display the first 10 rows of the DataFrame
display(ad_df.head(10))

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict

nltk.download('all')


In [ ]:
# Process each transcript
import numpy as np
if INDUSTRY_SPECIFIC_AWARENESS:
    for idx, row in ad_df.iterrows():
        transcript = row['transcript']
        product_cat_keyword_similarities = {}
        for keyword in row['product_cat_keywords'][1:-1].replace("'", "").split(", "):
            similarity = round(float(ta.get_semantic_similarity(transcript, keyword)), 3)
            product_cat_keyword_similarities[keyword] = similarity
        
        sorted_keywords = sorted(product_cat_keyword_similarities.items(), key=lambda x: x[1], reverse=True)
        top_3_keywords = sorted_keywords[:3]
        top_3_average = round(float(np.mean([sim for _, sim in top_3_keywords])), 3)
        
        print(f"\nTranscript {row['commercial_number']}:")
        print(f"Top 3 keywords:")
        for keyword, similarity in top_3_keywords:
            print(f"- {keyword}: {similarity}")
        print(f"Top 3 average similarity: {top_3_average}")
        
        ad_df.at[idx, 'product_cat_keyword_similarity'] = top_3_average
        ad_df.at[idx, 'product_cat_top_keywords'] = ', '.join([keyword for keyword, _ in top_3_keywords])
if BRAND_SPECIFIC_AWARENESS:
    for idx, row in ad_df.iterrows():
        transcript = row['transcript']
        product_brand_keyword_similarities = {}
        
        for keyword in row['product_brand_keywords'][1:-1].replace("'", "").split(", "):
            similarity = round(float(ta.get_semantic_similarity(transcript, keyword)), 3)
            product_brand_keyword_similarities[keyword] = similarity
        
        sorted_keywords = sorted(product_brand_keyword_similarities.items(), key=lambda x: x[1], reverse=True)
        top_3_keywords = sorted_keywords[:3]
        top_3_average = round(float(np.mean([sim for _, sim in top_3_keywords])), 3)
        
        print(f"\nTranscript {row['commercial_number']}:")
        print(f"Top 3 brand keywords:")
        for keyword, similarity in top_3_keywords:
            print(f"- {keyword}: {similarity}")
        print(f"Top 3 average brand similarity: {top_3_average}")
        
        ad_df.at[idx, 'product_brand_keyword_similarity'] = top_3_average
        ad_df.at[idx, 'product_brand_top_keywords'] = ', '.join([keyword for keyword, _ in top_3_keywords])

In [ ]:

ad_df.to_csv('csvs/4_ad_df_keywords.csv', index=False)

In [ ]:
import pandas as pd
ad_df = pd.read_csv('csvs/4_ad_df_keywords.csv')

In [ ]:


ad_df[ad_df.isnull().any(axis=1)].head()
ad_df[ad_df.isna().any(axis=1)].head()

ad_df = ad_df[ad_df['transcript'] != '']
ad_df = ad_df[ad_df['transcript'] != '']

# print all from ad_df with empty values
display(ad_df[ad_df.isnull().any(axis=1)])
display(ad_df[ad_df.isna().any(axis=1)])

In [ ]:
# INFO: Use for undersampling
# min_samples = min(len(ad_df[ad_df['BDM'] == 1]), len(ad_df[ad_df['BDM'] == 0]))
# ad_df_balanced = pd.concat([
#     ad_df[ad_df['BDM'] == 1].sample(n=min_samples, random_state=42),
#     ad_df[ad_df['BDM'] == 0].sample(n=min_samples, random_state=42)
# ]).reset_index(drop=True)
# print(f"Total rows: {len(ad_df_balanced)}")

# ad_df_balanced = ad_df

In [ ]:
print(f"Rows with BDM = 1.0: {len(ad_df[ad_df['BDM'] == 1.0])}")
print(f"Rows with BDM = 0.0: {len(ad_df[ad_df['BDM'] == 0.0])}")

In [ ]:
commercial_numbers = ad_df['commercial_number']


## Ansatz 1 (Machine learning)

In [ ]:
# Print available columns
print("Available columns:")
print(ad_df.columns.tolist())

In [ ]:
ad_df.to_csv('csvs/5_ad_df.csv', index=False)

In [ ]:
import pandas as pd
ad_df = pd.read_csv('csvs/5_ad_df.csv')

In [ ]:
import models as m

data = m.prepare_model_data(ad_df, INDUSTRY_SPECIFIC_AWARENESS, BRAND_SPECIFIC_AWARENESS)
target = ad_df['BDM']
base_models = m.get_base_models()
param_distributions = m.get_param_distributions()
tuned_models = m.tune_models(data, target, base_models, param_distributions)

trained_models = m.train_models(data, target, tuned_models, INDUSTRY_SPECIFIC_AWARENESS, BRAND_SPECIFIC_AWARENESS)

# Evaluate the trained models
results_df, predictions = m.evaluate_models(data, target, trained_models)

original_data = ad_df.copy()
commercial_number = original_data['commercial_number']
display(original_data.head(10))

original_data = pd.concat([original_data, commercial_number], axis=1)

original_data = pd.concat([original_data, predictions], axis=1)

# After your existing model training code
m.display_model_results(data, target, trained_models, results_df)


predicted_data = original_data

# only include the top 3 models prediction results
predicted_data = predicted_data[['commercial_number', 'BDM', 'Logistic Regression_result', 'Random Forest_result', 'Support Vector Machine_result']]
# write the majority result of the colums logistic regression, random forest and support vector machine to a new column majority vote
predicted_data['majority_vote'] = predicted_data[['Logistic Regression_result', 'Random Forest_result', 'Support Vector Machine_result']].mode(axis=1)[0]
display(predicted_data.head(10))


m.analyze_decision_tree(data, target, tuned_models)
